# --- Day 24: Never Tell Me The Odds ---
It seems like something is going wrong with the snow-making process. Instead of forming snow, the water that's been absorbed into the air seems to be forming hail!

Maybe there's something you can do to break up the hailstones?

Due to strong, probably-magical winds, the hailstones are all flying through the air in perfectly linear trajectories. You make a note of each hailstone's position and velocity (your puzzle input). For example:
```
19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
```
Each line of text corresponds to the position and velocity of a single hailstone. The positions indicate where the hailstones are right now (at time 0). The velocities are constant and indicate exactly how far each hailstone will move in one nanosecond.

Each line of text uses the format px py pz @ vx vy vz. For instance, the hailstone specified by 20, 19, 15 @ 1, -5, -3 has initial X position 20, Y position 19, Z position 15, X velocity 1, Y velocity -5, and Z velocity -3. After one nanosecond, the hailstone would be at 21, 14, 12.

Perhaps you won't have to do anything. How likely are the hailstones to collide with each other and smash into tiny ice crystals?

To estimate this, consider only the X and Y axes; ignore the Z axis. Looking forward in time, how many of the hailstones' paths will intersect within a test area? (The hailstones themselves don't have to collide, just test for intersections between the paths they will trace.)

In this example, look for intersections that happen with an X and Y position each at least 7 and at most 27; in your actual data, you'll need to check a much larger test area. Comparing all pairs of hailstones' future paths produces the following results:
```
Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 18, 19, 22 @ -1, -1, -2
Hailstones' paths will cross inside the test area (at x=14.333, y=15.333).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 20, 25, 34 @ -2, -2, -4
Hailstones' paths will cross inside the test area (at x=11.667, y=16.667).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=6.2, y=19.4).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for hailstone A.

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 20, 25, 34 @ -2, -2, -4
Hailstones' paths are parallel; they never intersect.

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=-6, y=-5).

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for both hailstones.

Hailstone A: 20, 25, 34 @ -2, -2, -4
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=-2, y=3).

Hailstone A: 20, 25, 34 @ -2, -2, -4
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for hailstone B.

Hailstone A: 12, 31, 28 @ -1, -2, -1
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for both hailstones.
```
So, in this example, 2 hailstones' future paths cross inside the boundaries of the test area.

However, you'll need to search a much larger test area if you want to see if any hailstones might collide. Look for intersections that happen with an X and Y position each at least 200000000000000 and at most 400000000000000. Disregard the Z axis entirely.

Considering only the X and Y axes, check all pairs of hailstones' future paths for intersections. How many of these intersections occur within the test area?

In [27]:
from utilities import get_lines
from collections import namedtuple
from itertools import combinations

In [2]:
SOURCE = 'sample'

In [3]:
lines = get_lines(SOURCE)
if SOURCE == 'input':
    MIN, MAX = 200000000000000, 400000000000000
else:
    MIN, MAX = 7, 27

In [4]:
lines[0]

'19, 13, 30 @ -2,  1, -2'

In [5]:
len(lines)

5

In [6]:
Hail = namedtuple('Hail', 'x y z dx dy dz b m')

In [7]:
def get_b(x, y, dx, dy):
    b = y-(dy/dx)*x
    return b

In [8]:
get_b(19,13,-2,1)

22.5

In [9]:
def initialize(lines):
    hailstorm = list()
    for line in lines:
        pt, vel = line.split('@')
        p = [int(p.strip()) for p in pt.split(',')]
        v = [int(v.strip()) for v in vel.split(',')]
        b = get_b(p[0],p[1],v[0],v[1])
        m = v[1]/v[0]
        hailstorm.append(Hail(*p,*v,b,m))
    return hailstorm

In [10]:
def is_valid(hail):
    if (hail.x > MAX)&(hail.dx>=0):
        return False
    if (hail.y > MAX)&(hail.dy>=0):
        return False
    if (hail.x < MIN)&(hail.dx<=0):
        return False
    if (hail.y < MIN)&(hail.dy<=0):
        return False
    return True

In [11]:
def get_valid_hail(hailstorm):
    for hail in hailstorm:
        if is_valid(hail)==False:
            hailstorm.remove(hail)
    return hailstorm

In [12]:
hailstorm = initialize(lines)
print(len(hailstorm))
# hailstorm = get_valid_hail(hailstorm)
# print(len(hailstorm))

5


In [13]:
def get_x(hail0, hail1):
    if hail0.m==hail1.m:
        return float('inf')
    x = (hail1.b-hail0.b)/(hail0.m-hail1.m)
    return x

In [14]:
def get_xy(hail0, hail1):
    x = get_x(hail0, hail1)
    y = hail0.m*x + hail0.b
    return x, y

In [28]:
hail_pairs = list(combinations(hailstorm,2))
len(hail_pairs)

10

In [29]:
def get_intersection(hail_pair):
    h0, h1 = hail_pair
    return get_xy(h0, h1)   

In [30]:
def check_intersection(x,y):
    if (x>=MIN)&(x<=MAX)&(y>=MIN)&(y<=MAX):
        return True
    return False

In [31]:
print(get_intersection(hail_pairs[0]))
print(check_intersection(*get_intersection(hail_pairs[0])))

(14.333333333333334, 15.333333333333332)
True


Note to self: you too many! intersections in the past are still here.

In [32]:
intersecting_pairs = list()
for pair in hail_pairs:
    x, y = get_intersection(pair)
    if check_intersection(x,y):
        intersecting_pairs.append(pair)
        print(pair)
        print(x,y,'\n')
len(intersecting_pairs)

(Hail(x=19, y=13, z=30, dx=-2, dy=1, dz=-2, b=22.5, m=-0.5), Hail(x=18, y=19, z=22, dx=-1, dy=-1, dz=-2, b=1.0, m=1.0))
14.333333333333334 15.333333333333332 

(Hail(x=19, y=13, z=30, dx=-2, dy=1, dz=-2, b=22.5, m=-0.5), Hail(x=20, y=25, z=34, dx=-2, dy=-2, dz=-4, b=5.0, m=1.0))
11.666666666666666 16.666666666666668 

(Hail(x=19, y=13, z=30, dx=-2, dy=1, dz=-2, b=22.5, m=-0.5), Hail(x=20, y=19, z=15, dx=1, dy=-5, dz=-3, b=119.0, m=-5.0))
21.444444444444443 11.777777777777779 

(Hail(x=18, y=19, z=22, dx=-1, dy=-1, dz=-2, b=1.0, m=1.0), Hail(x=20, y=19, z=15, dx=1, dy=-5, dz=-3, b=119.0, m=-5.0))
19.666666666666668 20.666666666666668 

(Hail(x=20, y=25, z=34, dx=-2, dy=-2, dz=-4, b=5.0, m=1.0), Hail(x=20, y=19, z=15, dx=1, dy=-5, dz=-3, b=119.0, m=-5.0))
19.0 24.0 



5